In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#for modelling
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV

from xgboost import XGBRegressor
import warnings



In [3]:
df=pd.read_csv('data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
X=df.drop(columns=['math_score'],axis=1)


In [5]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [6]:
Y=df['math_score']
Y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [7]:
num_features =X.select_dtypes(exclude=['object']).columns
num_features

Index(['reading_score', 'writing_score'], dtype='object')

In [8]:
cat_features=X.select_dtypes(include=['object']).columns
cat_features

Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')

In [9]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer([
    ('OneHotEncoder',oh_transformer,cat_features),
    ('StandardScaler',numerical_transformer,num_features)
])



In [10]:
X=preprocessor.fit_transform(X)

In [11]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=42)
X_train.shape,X_test.shape

((700, 19), (300, 19))

In [12]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2score=r2_score(true,predicted)

    return mae,mse,rmse,r2score

In [15]:
models={
    "LinearRegression": LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "SVR": SVR(),
    "AdaBoostClassifier":AdaBoostClassifier()
}

models_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,Y_train)

    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    model_train_mae,model_train_mse,model_train_rmse,model_train_r2score=evaluate_model(y_train_pred,Y_train)
    model_test_mae,model_test_mse,model_test_rmse,model_test_r2score=evaluate_model(Y_test,y_test_pred)

    print(list(models.keys())[i])
    models_list.append(list(models.keys())[i])    

    print('Model performance on training data')
    print("MAE: ",model_train_mae)
    print("MSE: ",model_train_mse)
    print("RMSE: ",model_train_rmse)
    print("r2_score: ",model_train_r2score)
    print()

    print('Model performance on test data')
    print("MAE: ",model_test_mae)
    print("MSE: ",model_test_mse)
    print("RMSE: ",model_test_rmse)
    print("r2_score: ",model_test_r2score)

    r2_list.append(model_test_r2score)

LinearRegression
Model performance on training data
MAE:  4.2075
MSE:  27.745535714285715
RMSE:  5.267403128134936
r2_score:  0.8591343509173388

Model performance on test data
MAE:  4.477916666666666
MSE:  31.730989583333333
RMSE:  5.633026680509629
r2_score:  0.8724693130062864
DecisionTreeRegressor
Model performance on training data
MAE:  0.02142857142857143
MSE:  0.08928571428571429
RMSE:  0.2988071523335984
r2_score:  0.9995950762043908

Model performance on test data
MAE:  6.623333333333333
MSE:  68.01
RMSE:  8.246817568007675
r2_score:  0.7266595799143266
RandomForestRegressor
Model performance on training data
MAE:  1.8257964285714283
MSE:  5.388919508928572
RMSE:  2.3214046413601768
r2_score:  0.973301707541374

Model performance on test data
MAE:  4.876244444444445
MSE:  37.848934685185185
RMSE:  6.152148786008445
r2_score:  0.8478805512918139
Lasso
Model performance on training data
MAE:  5.158226125142767
MSE:  42.38746971802188
RMSE:  6.510566005964603
r2_score:  0.6910901

c:\Users\SAKSHAM\Desktop\mlproject\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier
Model performance on training data
MAE:  8.992857142857142
MSE:  131.25285714285715
RMSE:  11.456563932648269
r2_score:  -1.7726618923384319

Model performance on test data
MAE:  9.66
MSE:  145.14666666666668
RMSE:  12.047683041426126
r2_score:  0.41663798205114144


In [17]:
pd.DataFrame(list(zip(models_list,r2_list)),columns=['Model Name','R2 Score']).sort_values(by='R2 Score',ascending=False)

,Model Name,R2 Score
4,Ridge,0.875907
0,LinearRegression,0.872469
2,RandomForestRegressor,0.847881
3,Lasso,0.810281
5,KNeighborsRegressor,0.772328
6,SVR,0.726848
1,DecisionTreeRegressor,0.726660
7,AdaBoostClassifier,0.416638
